In [1]:
import tensorflow as tf
import os
import numpy as np
import re
import pandas as pd

http://ai.stanford.edu/~amaas/data/sentiment/

In [14]:
vocab_imdb = open("aclImdb/vocab", "r").read().split("\n")
vocab_imdb.insert(0, "<PAD>")
vocab_imdb.insert(1, "<UNK>")

MAX_LEN = 400
data_dir = "aclImdb/"

with open(data_dir + "train/pos/12_9.txt", "r") as sample:
    print(sample.read())

Liked Stanley & Iris very much. Acting was very good. Story had a unique and interesting arrangement. The absence of violence and sex was refreshing. Characters were very convincing and felt like you could understand their feelings. Very enjoyable movie.


In [3]:
def process_str(string):
    # string = string.strip().lower()
    string = tf.regex_replace(string, r"[^A-Za-z0-9(),!?\'\`]", " ")
    string = tf.regex_replace(string, r"\'s", " \'s")
    string = tf.regex_replace(string, r"\'ve", " \'ve")
    string = tf.regex_replace(string, r"n\'t", " n\'t")
    string = tf.regex_replace(string, r"\'re", " \'re")
    string = tf.regex_replace(string, r"\'d", " \'d")
    string = tf.regex_replace(string, r"\'ll", " \'ll")
    string = tf.regex_replace(string, r",", " , ")
    string = tf.regex_replace(string, r"!", " ! ")
    string = tf.regex_replace(string, r"\s{2,}", " ")
    # Truncate string
    string = tf.string_split([string]).values[0:MAX_LEN]
    return string

In [4]:
def load_dataset(data_dir, words, part="train"):
    # read file names
    pos = os.listdir(data_dir + part + "/pos")
    pos = [data_dir + part + "/pos/" + f for f in pos]
    neg = os.listdir(data_dir + part + "/neg")
    neg = [data_dir + part + "/neg/" + f for f in neg]
    files = pos + neg
    labels = np.concatenate((np.ones(12500), np.zeros(12500)))
    # Create shuffling indices
    randomize = np.arange(len(labels))
    np.random.shuffle(randomize)
    # Shuffle the two arrays in unison
    files = np.array(files)[randomize]
    labels = labels[randomize]
    sentences = tf.data.TextLineDataset(files)
    labels = tf.data.Dataset.from_tensor_slices(labels)
    labels = labels.map(lambda x: tf.cast(x, tf.int32))
    sentences = sentences.map(lambda line: process_str(line)).map(lambda tokens: (words.lookup(tokens)))
    sizes = sentences.map(lambda x: tf.size(x))
    return tf.data.Dataset.zip((sentences, labels, sizes))

In [5]:
with tf.device("/cpu:0"), tf.name_scope("data"):
    words = tf.contrib.lookup.index_table_from_tensor(mapping=tf.constant(vocab_imdb), default_value=1)

    train = load_dataset(data_dir, words, "train").shuffle(buffer_size=256).padded_batch(128, padded_shapes=([None], [], [])).prefetch(1)
    test = load_dataset(data_dir, words, "test").shuffle(buffer_size=256).padded_batch(128, padded_shapes=([None], [], [])).prefetch(1)

    iterator = tf.data.Iterator.from_structure(train.output_types,
                                           train.output_shapes)
    train_iterator = iterator.make_initializer(train)
    test_iterator = iterator.make_initializer(test)

    doc, sentiment, size = iterator.get_next()
    doc = tf.cast(doc, tf.int32)
    sentiment = tf.cast(sentiment, tf.float32)
    sentiment = tf.reshape(sentiment, [-1, 1])

Instructions for updating:
Use the retry module or similar alternatives.


In [6]:
with tf.device("/cpu:0"), tf.name_scope("embedding"):
    embedding = tf.get_variable("embedding", [len(vocab_imdb) + 1, 128], initializer=tf.glorot_uniform_initializer())
    embedded_data = tf.nn.embedding_lookup(embedding, doc)
    embedded_data_dropout = tf.nn.dropout(embedded_data, 0.5)

In [7]:
with tf.name_scope("LSTM"):
    lstm_cell = tf.nn.rnn_cell.LSTMCell(128, initializer=tf.glorot_uniform_initializer())
    lstm_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_cell, input_keep_prob=0.5, output_keep_prob=0.5, state_keep_prob=0.5)


In [8]:
with tf.name_scope("Outputs"):
    outputs, states = tf.nn.dynamic_rnn(lstm_cell, embedded_data, dtype=tf.float32, sequence_length=size)
    net = tf.layers.flatten(outputs)
    net.set_shape([doc.shape[0], 128 * MAX_LEN])
    dropout = tf.nn.dropout(net, keep_prob=0.5)
    logits = tf.layers.dense(dropout, 1, activation=None,
        kernel_initializer=tf.glorot_uniform_initializer())

In [9]:
with tf.name_scope("Loss"):
    cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=sentiment)
    loss = tf.reduce_mean(cross_entropy)

In [10]:
with tf.name_scope("Optimizer"):
    adam = tf.train.AdamOptimizer(learning_rate=0.0001)
    optimizer = adam.minimize(loss)
    grads_and_vars = adam.compute_gradients(loss)

In [11]:
# How many example equals the results
correct_preds = tf.equal(tf.to_float(tf.nn.sigmoid(logits) > 0.5), sentiment)

# Calculate mini batch accuracy
accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))


In [12]:
with tf.name_scope("Summaries"):
    acc_summary = tf.summary.scalar("Accuracy", accuracy)
    loss_summary = tf.summary.scalar("Loss", loss)
    merged = tf.summary.merge_all()

In [13]:
with tf.Session() as sess:
    words.init.run()
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('./graphs/imdb_lstm', sess.graph)
    for i in range(15):
        sess.run(train_iterator)

        try:
            total_loss = []
            while True:
                _, l, summary = sess.run([optimizer, loss, merged])
                total_loss.append(l)
                writer.add_summary(summary, i)
                writer.flush()
        except tf.errors.OutOfRangeError as e:
            pass

        sess.run(train_iterator)

        try:
            total_accuracy = []
            while True:
                acc = sess.run([accuracy])
                total_accuracy.append(acc)
        except tf.errors.OutOfRangeError as e:
            pass

        # Calculate epoch accuracy
        total_loss = np.mean(total_loss)
        total_accuracy = np.mean(total_accuracy)
        print("Epoch {0:.2f}: Loss {0:.2f} Accuracy {0:.2f}".format(i, total_loss, total_accuracy))
        sess.run(test_iterator)
        total_accuracy_test = []
        try:
            while True:
                acc = sess.run(accuracy)
                total_accuracy_test.append(acc)
        except tf.errors.OutOfRangeError as e:
            total_accuracy_test = np.mean(total_accuracy_test)
            print("Accuracy over test data: {0:.2f}".format(total_accuracy_test))
        


    writer.close()


Epoch 0: Loss 0.6283934712409973 Accuracy 0.8471619486808777
Accuracy over test data: 0.822664201259613
Epoch 1: Loss 0.3458918333053589 Accuracy 0.9133290648460388
Accuracy over test data: 0.855046272277832
Epoch 2: Loss 0.24065570533275604 Accuracy 0.9370056986808777
Accuracy over test data: 0.8547114133834839
Epoch 3: Loss 0.1829719990491867 Accuracy 0.9547114372253418
Accuracy over test data: 0.8515146970748901


KeyboardInterrupt: 